# Challenge 2
An important aspect of pragmatic vector space methods is the ability to handle vectors and matrices.
A large collection of linear algebra functions is available in [SciPy.linalg](https://docs.scipy.org/doc/scipy/reference/linalg.html).
These functions can be employed in conjunction with the tools available in [NumPy](http://www.numpy.org/).
We note that the main object in NumPy is the homogeneous multidimensional array.

## Matrix
We begin by creating a simple matrix.
One possible approach to complete this task is to use ```scipy.linalg.circulant(c)```.

In [44]:
from scipy.linalg import circulant
my_circ_matrix = circulant([1, 2, 3])

Alternatively, you can construct the familiar discrete Fourier transform matrix with ```scipy.linalg.dft(n)```.

In [45]:
from scipy.linalg import dft
my_dft_matrix = dft(3)

The inverse of a matrix can be computed using ```scipy.linalg.inv(a)```.

In [46]:
from scipy.linalg import inv
my_idft_matrix = inv(my_dft_matrix)

The operation ```numpy.dot(a, b)``` computes the dot product of two arrays.
For 2-D arrays it is equivalent to matrix multiplication, and for 1-D arrays to inner product of vectors (without complex conjugation).

In [47]:
import numpy as np
matrix_prod1 = np.dot(my_dft_matrix, my_circ_matrix)
matrix_prod2 = np.dot(matrix_prod1, my_idft_matrix)

np.set_printoptions(suppress=True)
print(matrix_prod2)

[[ 10.+0.j   0.+0.j   0.+0.j   0.-0.j]
 [ -0.-0.j  -2.+2.j   0.-0.j   0.+0.j]
 [ -0.-0.j   0.+0.j  -2.+0.j   0.-0.j]
 [  0.-0.j   0.+0.j   0.+0.j  -2.-2.j]]


### Questions
These steps and their solutions immediately bring up three questions.
 * Are circulant matrices always diagonalized by the discrete Fourier transform matrix and its inverse?
 * Are product of circulant matrices (of a same size) always circulant matrices?
 * Do all pairs of circulant matrices commute under matrix multiplication?

## Determinant
The determinant of a square matrix is a value derived arithmetically from the coefficients of the matrix, and it summarizes a multivariable phenomenon with a single number.
It can be computed with ```scipy.linalg.det(a)```.

In [72]:
from scipy.linalg import det
det(my_circ_matrix)

-160.0

The code below demonstrates how to create a function in Python, how to vectorize a function so that it can be applied to the elements of a matrix, and how to use ```random```.

In [73]:
import math
from numpy import random

def my_log(x):
    return math.log(x)

my_vec_log = np.vectorize(my_log)

A_step1 = my_vec_log(my_circ_matrix) # Numpy already offers a vectorized natural logarithm.
#A_step1 = np.log()

max_index = 10000
my_identity = np.identity(len(A_step1))
current_value = 0.0
for my_index in range(0, max_index):
    permutation_matrix = random.permutation(my_identity)
    sign_permuation = det(permutation_matrix)  
    current_value += sign_permuation*(np.exp(np.trace(np.dot(A_step1, permutation_matrix))))
a_step2 = math.factorial(len(A_step1)) * current_value / max_index
print(a_step2)

-161.9232


### Questions
It appears that the output of the loop above is close to the determinant of the circulant matrix ```my_circ_matrix```.
 * Go through the code and provide a compelling explain explanation of why these numbers are close.
 * Is this a property of circulant matrices, or would this finding extend to arbitrary matrices over the real numbers?

### Tasks
 * Build code to explore the fact that the determinant function is multiplicative: $\mathrm{det}(AB) = \mathrm{det}(A) \mathrm{det}(B)$.

## Solution! 

### Matrix Questions
* Circulant matrices are always diagonalized by the DFT matrix and its inverse. Without getting too deep, solving for the eigenvalues of some circulant matrix $C$ gives the DFT of the top row of $C$ and a matrix $U$ composed of eigenvectors (which also happens to be the unitary DFT matrix). This is true for any $C$. So we have something like 
$$C = U(I\lambda)U^*$$ and 
$$(I\lambda) = U^*CU$$

In [33]:
from scipy.linalg import circulant
from scipy.linalg import dft
from scipy.linalg import inv
import numpy as np

#n = 3
C = circulant([1,2,3])

F = dft(3)
Ucc = (3**-0.5)*F

Finv  = inv(F)
U = (3**0.5)*Finv

p1 = np.dot(Ucc, C)
p1 = np.dot(p1, U)

p2 = np.dot(U, p1)
p2 = np.dot(p2, Ucc)

np.set_printoptions(suppress=True)
print(p1)
print(p2)

[[ 6.0-0.j         0.0+0.j         0.0+0.j       ]
 [-0.0-0.j        -1.5+0.8660254j -0.0-0.j       ]
 [ 0.0-0.j         0.0-0.j        -1.5-0.8660254j]]
[[ 1.-0.j  3.-0.j  2.+0.j]
 [ 2.+0.j  1.+0.j  3.-0.j]
 [ 3.-0.j  2.-0.j  1.-0.j]]


No proof asked for so:
* Products of circulant matrices (of a same size) are always circulant matrices.
* All pairs of circulant matrices do commute under matrix multiplication.

### Determinant Questions

* Whats happening: We have something like
$$\frac{n!}{MAX} \sum_{i=1}^{MAX} \mathrm{det}(P_i)\cdot e^{tr(ln(C)\cdot P_i))}$$<br><br> (sum is displaying incorrectly on my browser...) where P is a random permutation. Compare this to the formulation of the determinant (the determinant can be expressed as a sum of products of entries of the matrix where each product has [MAX] terms and the coefficient of each product is −1 or 1... via Wikipedia). We take the log of each value in C, then multiply by P so we get randomized values on the diagonal. Sum those values and put that above e. Multiply e by a coefficiant of det(P) which will be either -1 or 1. The natural log business is because $ln(x+y) = ln(x)ln(y)$ allowing us to take the trace instead of multiplying, putting it back above e undoes the damage. Do this a huge number of times, average that, and then multiply by n! which is the number of terms to be summed when taking the determinant. A sort of random determinant that becomes more accurate with more iterations. 
* This should work for any matrix that doesn't have zero terms. ln(0) = -inf which is no good.

### Task

In [85]:
from scipy.linalg import det 
import numpy as np

A = [[1,2,3],[4,10,6],[9,8,7]]
B = [[5,2,3],[7,5,11],[4,1,1]]
C = [[1,2,3,4],[3,4,10,5],[2,9,8,7],[14,5,5,1]]
D = [[7,5,11,4],[4,3,12,1],[4,1,1,3],[12,7,6,10]]


print("det(A): " + repr(det(A)))
print("det(B): " + repr(det(B)))
print("det(AB): " + repr(det(np.dot(A,B))))
print("det(BA): " + repr(det(np.dot(B,A))))
print("det(C): " + repr(det(C)))
print("det(D): " + repr(det(D)))
print("det(CD): " + repr(det(np.dot(C,D))))
print("det(DC): " + repr(det(np.dot(D,C))))

det(A): -100.00000000000001
det(B): 4.999999999999997
det(AB): -499.9999999999984
det(BA): -500.0000000000029
det(C): 1565.0
det(D): 41.00000000000001
det(CD): 64165.00000000014
det(DC): 64164.999999999476
